# Breast Cancer Detection

## Group 9 - Spring 22/23

In [11]:
#dependincies
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision.transforms import transforms
from torchvision import models
import torch
import torch.nn as nn
from torch.optim import Adam
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import torch.nn.functional as F
import random
from sklearn.metrics import confusion_matrix 
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score

# Deep Learning Approach

In [40]:
class CBIS_Dataset(Dataset):
    """
    Dataset class inhereted from PyTorch's Dataset
    attr:
    annotations_dir: dir of csv file with all metadata
    root_dir: root_dir :)
    abnoramility: {mass, calc}
    mamo_orientation: {MLO, CC} MLO-> front view, CC-> side view
    breast: {LEFT, RIGHT}
    image_type: {cropped images, full mammogram images, ROI mask images}
    """
    def __init__(self, 
                 annotations_dir, 
                 root_dir, 
                 abnormality='mass', 
                 mamo_orientation='all', 
                 breast='all', 
                 image_type='cropped images',
                 RGB=False,
                 blur_aug='weak',
                 perspective_aug='weak'):
        
        self.abnormality = abnormality
        self.mamo_orientation = mamo_orientation
        self.breast = breast
        self.image_type = image_type
        df = pd.read_csv(annotations_dir)
        self.annotations = self.subset_dataframe(df)
        self.root_dir = root_dir
        self.RGB = RGB
        self.blur_aug= blur_aug.lower()
        self.perspective_aug= perspective_aug.lower()
        
        IMG_WIDTH, IMG_HEIGHT = 256, 256
        
        transform = [transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
                    transforms.ToTensor(),
                    transforms.RandomHorizontalFlip(p=0),
                    transforms.RandomRotation(degrees=(0,0))]
                
        self.trasformation = transforms.Compose(transform)
        
    def subset_dataframe(self, df: pd.DataFrame)->pd.DataFrame:
        if (self.mamo_orientation=='all')&(self.breast=='all'):
            return df[(df.image_type==self.image_type)&
                    (df.abnormality==self.abnormality)].reset_index(drop=True)
        elif (self.mamo_orientation=='all'):
            return df[(df.image_type==self.image_type)&
                      (df.Breast==self.breast)&
                      (df.abnormality==self.abnormality)].reset_index(drop=True)
        elif (self.breast=='all'):
            return df[(df.image_type==self.image_type)&
                      (df.mamo_orientation==self.mamo_orientation)&
                      (df.abnormality==self.abnormality)].reset_index(drop=True)
        else:
            return df[(df.image_type==self.image_type)&
                      (df.Breast==self.breast)&
                      (df.abnormality==self.abnormality)&
                      (df.mamo_orientation==self.mamo_orientation)].reset_index(drop=True)
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        path = os.path.join(self.root_dir, self.annotations.iloc[index]['image_path'])
        sample = Image.open(path)
        
        if self.RGB:
            sample = self.trasformation(sample).repeat(3,1,1)
        else:
            sample = self.trasformation(sample)
        target_class = torch.tensor([int(self.annotations.iloc[index]['malignant'])]).to(torch.float32)
        
        #random guassian blur and noise augmentation
        #initialize with no effect
        blur, std, p_perspective = 0,0,0 #guassian blur and perspective control variable (random variable to control when augmentation is done)
        if self.blur_aug == 'weak':
            blur = np.random.choice([0,7])
            std = np.random.uniform(low=0, high=0.01)
        elif self.blur_aug == 'strong':
            blur = np.random.choice([7,15])
            std = np.random.uniform(low=0, high=0.8)
        elif self.blur_aug == 'none':
            blur, std = 0, 0
        else:
            raise RuntimeError(f"Augmentation {self.blur_aug} is not implemented! choose normal, strong, or none")
            
        if self.perspective_aug == 'weak':
            p_perspective = 0.3
        elif self.perspective_aug == 'strong':
            p_perspective = 0.7
        elif self.perspective_aug == 'none':
            p_perspective = 0
        else:
            raise RuntimeError(f"Augmentation {self.perspective_aug} is not implemented! choose normal, strong, or none")

        if blur:
            sample = transforms.Compose([transforms.GaussianBlur(kernel_size = blur)])(sample)

        #sample = sample +  np.random.normal(0, std, sample.shape) #this was too much noise so we dropped the idea
         
        sample = transforms.Compose([transforms.RandomPerspective(distortion_scale=0.5, p=p_perspective)])(sample)
        
        return (sample, target_class)
    
    def show_image(self, index):
        sample = self.__getitem__(index)[0]
        PIL_transform = transforms.ToPILImage()
        PIL_transform(sample).show()
    
    def train_test_split(self):
        self.train_idx = self.annotations[self.annotations['TrainTest']=='Training'].index.to_list()
        self.test_idx = self.annotations[self.annotations['TrainTest']=='Test'].index.to_list()
        
        return Subset(self, self.train_idx), Subset(self, self.test_idx)
    
    def train_val_test_split(self, val_ratio=0.2):
        self.train_idx = self.annotations[self.annotations['TrainTest']=='Training'].index.to_list()
        self.test_idx = self.annotations[self.annotations['TrainTest']=='Test'].index.to_list()
        
        self.val_idx = random.sample(self.train_idx, round(len(self.train_idx)*val_ratio))
        self.train_idx = list(set(self.train_idx) - set(self.val_idx))
        
        return Subset(self, self.train_idx), Subset(self, self.val_idx), Subset(self, self.test_idx)

In [13]:
class baseline_model(nn.Module): #alexnet
    """
    Baseline model which is a variant of AlexNet
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 64, 11, stride=4, padding=2)
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(3, stride=2)
        self.conv2 = nn.Conv2d(64, 192, 5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm2d(192)
        self.pool2 = nn.MaxPool2d(3, stride=2)
        self.conv3 = nn.Conv2d(192, 384, 3, stride=1, padding=1)
        #self.conv4 = nn.Conv2d(384, 256, 3, stride=1, padding=1)
        #self.conv5 = nn.Conv2d(256, 256, 3, stride=1, padding=1)
        self.pool3 = nn.MaxPool2d(3, 2)
        self.drop = nn.Dropout(0.5)
        self.fc1 = nn.Linear(384*7*7, 1024)#(256*7*7, 1024)
        self.drop2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(1024, 128)
        self.fc3 = nn.Linear(128, 1)        
        
    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = F.relu(self.conv3(x))
        #x = F.relu(self.conv4(x))
        #x = self.pool3(F.relu(self.conv5(x)))
        x = self.pool3(x)
        x = self.drop(x)
        x = x.view(-1, 384*7*7)
        x = F.relu(self.fc1(x))
        x = self.drop2(x)
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

csv_dir = 'Dataset/csv/annotations.csv'
root_dir = 'Dataset/'

Data = CBIS_Dataset(annotations_dir=csv_dir, root_dir=root_dir, RGB=True, image_type='full mammogram images', blur_aug='none', perspective_aug='none')

# Training

In [15]:
BATCH_SIZE = 32
EPOCH = 20
LEARNING_RATE = 3e-4 #if you know you know 

train, val, test = Data.train_val_test_split()
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val, batch_size=BATCH_SIZE, shuffle=True)

#mod = baseline_model().to(device)
#optimizer = Adam(mod.parameters(), lr=LEARNING_RATE)
#loss_f = nn.BCELoss()

mod = models.alexnet(pretrained=True).to(device)

for param in mod.features.parameters(): param.requires_grad=False
mod.classifier[-1] = nn.Linear(4096, 1)
mod = mod.to(device)

optimizer = Adam(mod.classifier.parameters(), lr=LEARNING_RATE)
loss_f = nn.BCEWithLogitsLoss()
min_valid_loss = np.inf

train_results = {'loss':[], 'f1':[], 'precision':[], 'recall': []}
val_results = {'loss':[], 'f1':[], 'precision':[], 'recall': []}

for epoch in range(EPOCH):
    train_loss, f1_train, recall_train, precision_train = 0.0, 0.0, 0.0, 0.0
    for batch in train_loader:
        x,y = batch
        x,y = x.to(device), y.to(device)
        pred = mod(x)
        loss = loss_f(pred, y)
        
        f1_train_temp = f1_score(y.to('cpu').detach(), torch.sigmoid(pred).round().to('cpu').detach())
        recall_train_temp = recall_score(y.to('cpu').detach(), torch.sigmoid(pred).round().to('cpu').detach(), zero_division=0)
        precision_train_temp = precision_score(y.to('cpu').detach(), torch.sigmoid(pred).round().to('cpu').detach(), zero_division=0)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del x
        del y
        if device == 'cuda': torch.cuda.empty_cache()
        train_loss += loss.item()
        f1_train += f1_train_temp
        recall_train += recall_train_temp
        precision_train += precision_train_temp
            
    val_loss, f1_val, recall_val, precision_val = 0.0, 0.0, 0.0, 0.0
    mod.eval()    
    for val_batch in val_loader:
        x,y = val_batch
        x,y = x.to(device), y.to(device)
    
        pred_val = mod(x)
        loss = loss_f(pred_val, y)
        
        f1_val_temp = f1_score(y.to('cpu').detach(), torch.sigmoid(pred_val).round().round().to('cpu').detach())
        recall_val_temp = recall_score(y.to('cpu').detach(), torch.sigmoid(pred_val).round().round().to('cpu').detach(), zero_division=0)
        precision_val_temp = precision_score(y.to('cpu').detach(), torch.sigmoid(pred_val).round().round().to('cpu').detach() , zero_division=0)
        
        del x
        del y
        if device == 'cuda': torch.cuda.empty_cache()
        val_loss += loss.item()
        f1_val += f1_val_temp
        recall_val += recall_val_temp
        precision_val += precision_val_temp
        
    print(f'Epoch {epoch+1} \nTraining Loss: {train_loss / len(train_loader)} \tValidation Loss: {val_loss / len(val_loader)}')
    print(f'Training F1: {f1_train/len(train_loader)} \tValidation F1: {f1_val/ len(val_loader)} \
          \nTraining Recall: {recall_train/ len(train_loader)}, \tValidation Recall: {recall_val/len(val_loader)} \
          \nTraining Precision: {precision_train/ len(train_loader)}, \tValidation Precision: {precision_val/len(val_loader)}','\n', 30*'*')
    
    train_results['loss'].append(train_loss/len(train_loader))
    train_results['f1'].append(f1_train/len(train_loader))
    train_results['precision'].append(recall_train/len(train_loader))
    train_results['recall'].append(precision_train/len(train_loader))
    
    val_results['loss'].append(val_loss/len(val_loader))
    val_results['f1'].append(f1_val/len(val_loader))
    val_results['precision'].append(recall_val/len(val_loader))
    val_results['recall'].append(precision_val/len(val_loader))
     
    if min_valid_loss > val_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{val_loss:.6f}) \t Saving The Model')
        min_valid_loss = val_loss
         
        # Saving State Dict
        #torch.save(mod.state_dict(), 'saved_model.pth')

Epoch 1 
Training Loss: 0.7295196075593272 	Validation Loss: 0.703574612736702
Training F1: 0.35966679290592785 	Validation F1: 0.0           
Training Recall: 0.48185518097757185, 	Validation Recall: 0.0           
Training Precision: 0.35892621950771353, 	Validation Precision: 0.0 
 ******************************
Validation Loss Decreased(inf--->5.628597) 	 Saving The Model
Epoch 2 
Training Loss: 0.6918938390670284 	Validation Loss: 0.6705066785216331
Training F1: 0.375526453143109 	Validation F1: 0.6402393121143121           
Training Recall: 0.380322665394572, 	Validation Recall: 0.7249935919343814           
Training Precision: 0.4795690934593982, 	Validation Precision: 0.5888435990338163 
 ******************************
Validation Loss Decreased(5.628597--->5.364053) 	 Saving The Model
Epoch 3 
Training Loss: 0.6964101791381836 	Validation Loss: 0.6933934316039085
Training F1: 0.4329604627327457 	Validation F1: 0.16260822510822512           
Training Recall: 0.5618589614719645, 

# Validation set Results 

In [17]:
EXP_NAME = 'aug_horizontal'
i=0
for metric in train_results.keys():
    plt.plot(range(len(train_results[metric])), train_results[metric])
    plt.plot(range(len(val_results[metric])), val_results[metric])
    plt.title(metric)
    plt.legend(['train', 'validation'])
    #plt.show()
    plt.savefig(f'{EXP_NAME}{i}.jpeg')
    plt.close()
    i = i+1

# Retrain with Validation set to test model

In [18]:
BATCH_SIZE = 32
EPOCH = 30
LEARNING_RATE = 3e-4 #if you know you know 

train, test = Data.train_test_split()
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)

mod = models.alexnet(pretrained=True).to(device)

for param in mod.features.parameters(): param.requires_grad=False
mod.classifier[-1] = nn.Linear(4096, 1)
mod = mod.to(device)

optimizer = Adam(mod.classifier.parameters(), lr=LEARNING_RATE)
loss_f = nn.BCEWithLogitsLoss()

train_results = {'loss':[], 'f1':[], 'precision':[], 'recall': []}

for epoch in range(EPOCH):
    for batch in train_loader:
        x,y = batch
        x,y = x.to(device), y.to(device)
        pred = mod(x)
        loss = loss_f(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del x
        del y
        
        torch.save(mod.state_dict(), f'{EXP_NAME}saved_model.pth')
        
        if device == 'cuda': torch.cuda.empty_cache()
            

# Test set

In [20]:
y_pred_list = []
y_test = []
test_loader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)
mod.eval()
with torch.no_grad():
    for batch in test_loader:
        x,y = batch
        x,y = x.to(device), y.to(device)
        y_test_pred = torch.sigmoid(mod(x))
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())
        y_test.append(np.array(y.to('cpu')))

#y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [21]:
import json
y_test = np.vstack(y_test)
y_pred = np.vstack(y_pred_list)

f1_test = f1_train_temp = f1_score(y_test, y_pred)
recal_test = recall_train_temp = recall_score(y_test, y_pred, zero_division=0)
precision_test = precision_train_temp = precision_score(y_test, y_pred, zero_division=0)

print(EXP_NAME, '\nconfusion matrix:\n',confusion_matrix(y_pred, y_test), f'\nf1:{f1_test}\nrecall:{recal_test}\nprecision:{precision_test}') 

results_dict = {'confusion matrix': str(confusion_matrix(y_pred, y_test)),
               'f1': str(f1_test),
               'recall': str(recal_test),
               'precision': str(precision_test)}

with open(f'{EXP_NAME}results_dict.txt', 'w') as convert_file:
     convert_file.write(json.dumps(results_dict))

aug_horizontal 
confusion matrix:
 [[ 63  16]
 [155 129]] 
f1:0.6013986013986014
recall:0.8896551724137931
precision:0.45422535211267606


# SVM + PCA 

### Custom Functions

In [ ]:
### Returns transformed principal components
def train_test_transformed_PCA(train, test, components):
    # Train
    pca_train = PCA(n_components = components)
    pca_train.fit(train)
    transformed_data_train = pca_train.transform(train)
    print(sum(pca_train.explained_variance_ratio_))


    # Test
    pca_test = PCA(n_components = components)
    pca_test.fit(test)
    transformed_data_test = pca_test.transform(test)
    print(sum(pca_test.explained_variance_ratio_))
    
    return transformed_data_train, transformed_data_test

# function that you will use to convert matrix to dataframe, useful for visulization. 
def conf_matrix_to_df(conf_matrix, target_names):
    return pd.DataFrame(conf_matrix, columns=target_names, index=target_names)

## Pipeline to create 3 subsets of full alexnet

In [ ]:
# Make each model independent:

# 1. Full model
mod_full = models.alexnet(pretrained=True).to(device)
for param in mod_full.features.parameters(): param.requires_grad=False
mod_full.classifier[-1] = nn.Linear(4096, 1)
mod_full = mod_full.to(device)
mod_full.load_state_dict(torch.load('./results/horizontal_flipping/aug_horizontalsaved_model.pth', 
                    map_location=torch.device('cpu')))
mod_full.classifier = mod_full.classifier[:5:]
print(mod_full.classifier)

# 2. Truncated model (-last layer)
mod_trunc1 = models.alexnet(pretrained=True).to(device)
for param in mod_trunc1.features.parameters(): param.requires_grad=False
mod_trunc1.classifier[-1] = nn.Linear(4096, 1)
mod_trunc1.load_state_dict(torch.load('./results/horizontal_flipping/aug_horizontalsaved_model.pth', 
                    map_location=torch.device('cpu')))
mod_trunc1.classifier = mod_trunc1.classifier[:3:]
mod_trunc1 = mod_trunc1.to(device)
print(mod_trunc1.classifier)

## Fill separate model train/test subsets:

In [ ]:
BATCH_SIZE = 1
EPOCH = 1
# Collect Train/Test
train, test = Data.train_test_split()
train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=True)


########### Train ###########

# Features
dat_x_full_train = []
dat_x_trunc1_train = []
# Targets
dat_y_full_train = []
dat_y_trunc1_train = []


for epoch in range(EPOCH):
    for batch in train_loader:
        x,y = batch 
        x,y = x.to(device), y.to(device)
        # 1. Full model
        pred = mod_full(x)
        pred = np.array(pred.to('cpu').detach())
        y = np.array(y.to('cpu'))
        dat_x_full_train.append(pred)
        dat_y_full_train.append(y)
        
        # 2. Truncated model
        pred = mod_trunc1(x)
        pred = np.array(pred.to('cpu').detach())
        dat_x_trunc1_train.append(pred)
        dat_y_trunc1_train.append(y)
        
########### Test ###########

# Features
dat_x_full_test = []
dat_x_trunc1_test = []
# Targets
dat_y_full_test = []
dat_y_trunc1_test = []

for epoch in range(EPOCH):
    for batch in test_loader:
        x,y = batch #(,)
        x,y = x.to(device), y.to(device)
        # 1. Full model
        pred = mod_full(x)
        pred = np.array(pred.to('cpu').detach())
        y = np.array(y.to('cpu'))
        dat_x_full_test.append(pred)
        dat_y_full_test.append(y)
        
        # 2. Truncated model
        pred = mod_trunc1(x)
        pred = np.array(pred.to('cpu').detach())
        dat_x_trunc1_test.append(pred)
        dat_y_trunc1_test.append(y)
        

### Reshape train/test to numpy arrays:

In [ ]:
############## Train
# Features
x_full_train = np.array(dat_x_full_train).reshape(len(dat_x_full_train),  np.array(dat_x_full_train).shape[2])
x_trunc1_train = np.array(dat_x_trunc1_train).reshape(len(dat_x_trunc1_train), np.array(dat_x_trunc1_train).shape[2])
# Target
y_full_train = np.array(dat_y_full_train).reshape(len(dat_y_full_train), 1)
y_trunc1_train = np.array(dat_y_trunc1_train).reshape(len(dat_y_trunc1_train), 1)


############## Test
# Features
x_full_test = np.array(dat_x_full_test).reshape(len(dat_x_full_test), np.array(dat_x_full_test).shape[2])
x_trunc1_test = np.array(dat_x_trunc1_test).reshape(len(dat_x_trunc1_test), np.array(dat_x_trunc1_test).shape[2])

# Target
y_full_test = np.array(dat_y_full_test).reshape(len(dat_y_full_test), 1)
y_trunc1_test = np.array(dat_y_trunc1_test).reshape(len(dat_y_trunc1_test), 1)

print(x_full_train.shape)
print(x_trunc1_train.shape)
print(y_full_train.shape)
print(y_trunc1_train.shape)

print(x_full_test.shape)
print(x_trunc1_test.shape)
print(y_full_test.shape)
print(y_trunc1_test.shape)

### PCA and SVM Classification

In [ ]:
# PCA reduction for training/test (100 components)
x_full_train_tf, x_full_test_tf = train_test_transformed_PCA(x_full_train, x_full_test, 100)

# SVM using 100 PCs
svc = SVC(kernel = "poly")
svc.fit(x_full_train_tf, y_full_train.ravel())
predictions = svc.predict(x_full_test_tf)
conf_matrix = confusion_matrix(predictions, y_full_test)
print(conf_matrix)
print(classification_report(y_full_test, 
                      predictions))
print('Precision: %.3f' % precision_score(y_full_test, predictions))
print('Recall: %.3f' % recall_score(y_full_test, predictions))
print('F1: %.3f' % f1_score(y_full_test, predictions))
print('Accuracy: %.3f' % accuracy_score(y_full_test, predictions))

###  Truncated Model

In [ ]:
# PCA reduction for training/test (100 components)
x_trunc1_train_tf, x_trunc1_test_tf = train_test_transformed_PCA(x_trunc1_train, x_trunc1_test, 100)

# SVM using 100 PCs
svc = SVC(kernel = "poly")
svc.fit(x_trunc1_train_tf, y_trunc1_train.ravel())
predictions = svc.predict(x_trunc1_test_tf)
conf_matrix = confusion_matrix(predictions, y_trunc1_test)
print(conf_matrix)
print(classification_report(y_trunc1_test, 
                      predictions))

print('Precision: %.3f' % precision_score(y_trunc1_test, predictions))
print('Recall: %.3f' % recall_score(y_trunc1_test, predictions))
print('F1: %.3f' % f1_score(y_trunc1_test, predictions))
print('Accuracy: %.3f' % accuracy_score(y_trunc1_test, predictions))

### Just SVM

In [ ]:
############ Full
svc = SVC(kernel = "poly")
svc.fit(x_full_train, y_full_train.ravel())
predictions = svc.predict(x_full_test)
conf_matrix = confusion_matrix(predictions, y_full_test)
print(conf_matrix)
print(classification_report(y_full_test, 
                      predictions))

print('Precision: %.3f' % precision_score(y_full_test, predictions))
print('Recall: %.3f' % recall_score(y_full_test, predictions))
print('F1: %.3f' % f1_score(y_full_test, predictions))
print('Accuracy: %.3f' % accuracy_score(y_full_test, predictions))
############ Truncated 1

svc = SVC(kernel = "poly")
svc.fit(x_trunc1_train, y_trunc1_train.ravel())
predictions = svc.predict(x_trunc1_test)
conf_matrix = confusion_matrix(predictions, y_trunc1_test)
print(conf_matrix)
print(classification_report(y_trunc1_test, 
                      predictions))

print('Precision: %.3f' % precision_score(y_trunc1_test, predictions))
print('Recall: %.3f' % recall_score(y_trunc1_test, predictions))
print('F1: %.3f' % f1_score(y_trunc1_test, predictions))
print('Accuracy: %.3f' % accuracy_score(y_trunc1_test, predictions))